# ПОСТАНОВКА ЗАДАЧИ
В этом блоке мы разберем пример расчета рейтингов фильма, т. е. долей оценок внутри одного фильма. В итоге нам необходимо написать алгорим, показывающий для заданного фильма movieId распределение оценок: какая доля оценок пришлась на 0.5, на 1.0, ..., на 5.0.

При этом для расчета доли оценки нам надо знать сумму всех оценок каждого фильма. В разных системах подобный класс задачи называется оконными, кумулятивными функциями, партиционированием. Также они позволяют считать рейтинги внутри групп, бегущую сумму и другие полезные для аналитика метрики (специальные методы под эти задачи есть в Pandas). Мы рассмотрим один из них с использованием метода transform.

# СВОДНАЯ ТАБЛИЦА ПО НЕСКОЛЬКИМ ИЗМЕРЕНИЯМ
Решим задачу наиболее общим способом. Сначала построим сводную таблицу с необходимыми нам столбцами. Мы будем считать рейтинги для каждого фильма, поэтому в строках у нас должны идти пары movieId - rating. Сводная таблица для нескольких измерений строятся аналогично группировкам:

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 10)

data = pd.read_csv('./module05_files/ratings.csv')

data_pivot = data.pivot_table(index = ['movieId', 'rating'], values = 'timestamp', aggfunc = 'count').reset_index()

data_pivot.head(10)

,movieId,rating,timestamp
0,1,1.0,4
1,1,1.5,3
2,1,2.0,13
3,1,2.5,4
4,1,3.0,41
5,1,3.5,23
6,1,4.0,77
7,1,4.5,19
8,1,5.0,63
9,2,1.5,1


Теперь для каждого фильма movieId у нас есть сумма оценок для всех значений рейтинга от 0.5 до 5.0.

# СУММА РЕЙТИНГОВ ДЛЯ ФИЛЬМА
Теперь для каждого movieId надо как-то узнать сумму оценок и записать ее отдельным столбцом. И потом посчитать долю каждой оценки, просто поделив значение столбца timestamp на эту сумму. Для этого используем следующий прием: давайте сгруппируем датафрейм data_pivot по movieId, возьмем столбец timestamp и применим к нему lambda-функцию. Пока lambda-функция ничего делать не будет, просто возвращать значение строки:

In [9]:
data_pivot.groupby('movieId').timestamp.transform(lambda x: x)

0         4
1         3
2        13
3         4
4        41
         ..
28670     1
28671     1
28672     1
28673     1
28674     1
Name: timestamp, Length: 28675, dtype: int64

In [11]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [12]:
data_pivot['sum'] = (data_pivot
                     .groupby('movieId')
                     .timestamp
                     .transform(lambda x: sum(x)))

data_pivot.head(10)

,movieId,rating,timestamp,sum
0,1,1.0,4,247
1,1,1.5,3,247
2,1,2.0,13,247
3,1,2.5,4,247
4,1,3.0,41,247
5,1,3.5,23,247
6,1,4.0,77,247
7,1,4.5,19,247
8,1,5.0,63,247
9,2,1.5,1,107


Обратите внимание, что значение суммы рейтингов фильма с movieId=1 сохраняется во всех строках этого фильма.

# СЧИТАЕМ РАСПРЕДЕЛЕНИЕ РЕЙТИНГОВ
Теперь мы можем легко получить долю оценок каждого рейтинга для всех строк одного фильма:

In [13]:
data_pivot['share'] = data_pivot['timestamp'] / data_pivot['sum']

data_pivot.head(10)

,movieId,rating,timestamp,sum,share
0,1,1.0,4,247,0.016194
1,1,1.5,3,247,0.012146
2,1,2.0,13,247,0.052632
3,1,2.5,4,247,0.016194
4,1,3.0,41,247,0.165992
5,1,3.5,23,247,0.093117
6,1,4.0,77,247,0.311741
7,1,4.5,19,247,0.076923
8,1,5.0,63,247,0.255061
9,2,1.5,1,107,0.009346


# Упражнение
(1 возможный балл)
Чему равна сумма столбца share для movieId = 1?

In [14]:
data_pivot[data_pivot['movieId']==1]['share'].sum()

1.0

# ДОМАШНЕЕ ЗАДАНИЕ
В прошлом шаге мы получили распределение оценок каждого фильма для значений от 0.5 до 5.0. Такая разбивка весьма детальна, а пользователю смотреть такие оценки неудобно. Ваша задача написать алгоритм, который классифицирует эти оценки по более практичной шкале:

оценка 2 и меньше - низкий рейтинг
оценка 4 и меньше - средний рейтинг
оценка 4.5 и 5 - высокий рейтинг
Т. е. в итоге получить следующую таблицу:

    картинка =)

In [16]:
def range_rating(rating):
    if rating <= 2:
        return 'Низкий'
    if rating <= 4:
        return 'Средний'
    return 'Высокий'

In [18]:
data_pivot['class'] = data_pivot['rating'].map(range_rating)

In [19]:
data_pivot

,movieId,rating,timestamp,sum,share,class
0,1,1.0,4,247,0.016194,Низкий
1,1,1.5,3,247,0.012146,Низкий
2,1,2.0,13,247,0.052632,Низкий
3,1,2.5,4,247,0.016194,Средний
4,1,3.0,41,247,0.165992,Средний
...,...,...,...,...,...,...
28670,161944,5.0,1,1,1.000000,Высокий
28671,162376,4.5,1,1,1.000000,Высокий
28672,162542,5.0,1,1,1.000000,Высокий
28673,162672,3.0,1,1,1.000000,Средний


# Домашнее задание
(1 балл из 1)
Какую долю среди оценок фильма с movieId = 356 имеет 'средний рейтинг'? Ответ округлите до 2 знаков после запятой. Пример ответа: 0.32

In [36]:
data_pivot[(data_pivot['movieId']==356) & 
           (data_pivot['class']=='Средний')]['share'].sum()

0.5513196480938416